Dag 7

0 ->| Amp A |->| Amp B |->| Amp C |->| Amp D |->| Amp E |-> (to thrusters)

    
Input is (an integer from 0 to 4) Each phase setting is used exactly once

In [2]:
import pandas as pd
import numpy as np
from itertools import permutations

In [3]:
def parse_code(code):
    A=int(code/10000)
    B=int((code-A*10000)/1000)
    C=int((code-A*10000-B*1000)/100)
    opcode=int((code-A*10000-B*1000-C*100))
    return(A,B,C,opcode)

def read_ab(ic,i,B,C):
    a=ic[i+1] if C==1 else ic[ic[i+1]]
    b=ic[i+2] if B==1 else ic[ic[i+2]]
    return (a,b)

def Incode(ic,inn):
    output=[]
    j=i=0
    while i<len(ic):
        
        if ic[i]==99:
            break
        else:
            (A,B,C,opcode)=parse_code(ic[i])
            #print(f'opcode:{opcode},ic[i]:{ic[i]}')
            if opcode==1:
                (a,b)=read_ab(ic,i,B,C)
                if A:
                    ic[i+3]=a+b
                else:
                    ic[ic[i+3]]=a+b
                #print('a:{},b:{},a+b:{}'.format(a,b,a+b))
            elif opcode==2:
                (a,b)=read_ab(ic,i,B,C)
                if A:
                    ic[i+3]=a*b
                else:
                    ic[ic[i+3]]=a*b
                #print('a:{},b:{},a*b:{}'.format(a,b,a*b))
            elif opcode==3:
                if C:
                    ic[i+1]=inn[j]
                else:
                    ic[ic[i+1]]=inn[j]
                j+=1
                #print(f'in:{inn}, immode:{immode1,immode2,immode3},imic:{ic[i+1]},ic:{ic[ic[i+1]]}')
            elif opcode==4:
                if C:
                    output.append(ic[i+1])
                else:
                    output.append(ic[ic[i+1]])
                #print(f'out:{output}, immode:{immode1,immode2,immode3},imic:{ic[i+1]},ic:{ic[ic[i+1]]}')
            elif opcode==5:
                #if the first parameter is non-zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.
                (a,b)=read_ab(ic,i,B,C)
                if a!=0:
                    i=b
                else:
                    i+=3
                #print(f'a:{a}b:{b},C:{C},B:{B}')
            elif opcode==6:
                #if the first parameter is zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.
                (a,b)=read_ab(ic,i,B,C)
                if a==0:
                    i=b
                else:
                    i+=3
                #print(f'a:{a}b:{b},C:{C},B:{B}')
            elif opcode==7:
                #if the first parameter is less than the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.
                (a,b)=read_ab(ic,i,B,C)
                if a<b:
                    if A:
                        ic[i+3]=1
                    else:
                        ic[ic[i+3]]=1
                else: 
                    if A:
                        ic[i+3]=0
                    else:
                        ic[ic[i+3]]=0
                #print(f'a:{a}b:{b},C:{C},B:{B}')
            elif opcode==8:
                #if the first parameter is equal to the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.
                (a,b)=read_ab(ic,i,B,C)
                if a==b:
                    if A==1:
                        ic[i+3]=1
                    else:
                        ic[ic[i+3]]=1
                else: 
                    if A==1:
                        ic[i+3]=0
                    else:
                        ic[ic[i+3]]=0  
                #print(f'a:{a}b:{b},C:{C},B:{B}')
            else:
                print('error')
        if opcode in (1,2,7,8):
            i+=4
        elif opcode in (3,4):
            i+=2 
        
        #print(i)
        #print(output)
        #print('-----------------')
    return (output,ic)

In [4]:
class Amplifier:
    def __init__(self,ic):
        self.ic=ic
        self.i=0
        return
    
    def start_up(self,inn):    
        j=0
        while self.i<len(self.ic):
            if self.ic[self.i]==99:
                return 'Done'
            else:
                (A,B,C,opcode)=parse_code(self.ic[self.i])
                #print(f'opcode:{opcode} i:{self.i}')
                #print(self.ic)
                if opcode==1:
                    (a,b)=read_ab(self.ic,self.i,B,C)
                    if A:
                        self.ic[self.i+3]=a+b
                    else:
                        self.ic[self.ic[self.i+3]]=a+b
                elif opcode==2:
                    (a,b)=read_ab(self.ic,self.i,B,C)
                    if A:
                        self.ic[self.i+3]=a*b
                    else:
                        self.ic[self.ic[self.i+3]]=a*b
                elif opcode==3:
                    #print(inn)
                    if C:
                        self.ic[self.i+1]=inn[j]
                    else:
                        self.ic[self.ic[self.i+1]]=inn[j]
                    j+=1
                elif opcode==4:
                    if C:
                        self.i+=2
                        return self.ic[self.i-1]
                    else:
                        self.i+=2
                        return self.ic[self.ic[self.i-1]]
                elif opcode==5:
                    (a,b)=read_ab(self.ic,self.i,B,C)
                    if a!=0:
                        self.i=b
                    else:
                        self.i+=3
                elif opcode==6:
                    (a,b)=read_ab(self.ic,self.i,B,C)
                    if a==0:
                        self.i=b
                    else:
                        self.i+=3
                elif opcode==7:
                    (a,b)=read_ab(self.ic,self.i,B,C)
                    if a<b:
                        if A:
                            self.ic[self.i+3]=1
                        else:
                            self.ic[self.ic[self.i+3]]=1
                    else: 
                        if A:
                            self.ic[self.i+3]=0
                        else:
                            self.ic[self.ic[self.i+3]]=0
                elif opcode==8:
                    (a,b)=read_ab(self.ic,self.i,B,C)
                    if a==b:
                        if A==1:
                            self.ic[self.i+3]=1
                        else:
                            self.ic[self.ic[self.i+3]]=1
                    else: 
                        if A==1:
                            self.ic[self.i+3]=0
                        else:
                            self.ic[self.ic[self.i+3]]=0
                else:
                    print('error')
            if opcode in (1,2,7,8):
                self.i+=4
            elif opcode in (3,4):
                self.i+=2 
        return 'error'
    
    def update(self,inn):
        while self.i<len(self.ic):
            if self.ic[self.i]==99:
                return 'Done'
            else:
                (A,B,C,opcode)=parse_code(self.ic[self.i])
                #print(f'opcode:{opcode} i:{self.i}')
                #print(self.ic)
                if opcode==1:
                    (a,b)=read_ab(self.ic,self.i,B,C)
                    if A:
                        self.ic[self.i+3]=a+b
                    else:
                        self.ic[self.ic[self.i+3]]=a+b
                elif opcode==2:
                    (a,b)=read_ab(self.ic,self.i,B,C)
                    if A:
                        self.ic[self.i+3]=a*b
                    else:
                        self.ic[self.ic[self.i+3]]=a*b
                elif opcode==3:
                    #print(inn)
                    if C:
                        self.ic[self.i+1]=inn
                    else:
                        self.ic[self.ic[self.i+1]]=inn
                elif opcode==4:
                    if C:
                        self.i+=2
                        return self.ic[self.i-1]
                    else:
                        self.i+=2
                        return self.ic[self.ic[self.i-1]]
                elif opcode==5:
                    (a,b)=read_ab(self.ic,self.i,B,C)
                    if a!=0:
                        self.i=b
                    else:
                        self.i+=3
                elif opcode==6:
                    (a,b)=read_ab(self.ic,self.i,B,C)
                    if a==0:
                        self.i=b
                    else:
                        self.i+=3
                elif opcode==7:
                    (a,b)=read_ab(self.ic,self.i,B,C)
                    if a<b:
                        if A:
                            self.ic[self.i+3]=1
                        else:
                            self.ic[self.ic[self.i+3]]=1
                    else: 
                        if A:
                            self.ic[self.i+3]=0
                        else:
                            self.ic[self.ic[self.i+3]]=0
                elif opcode==8:
                    (a,b)=read_ab(self.ic,self.i,B,C)
                    if a==b:
                        if A==1:
                            self.ic[self.i+3]=1
                        else:
                            self.ic[self.ic[self.i+3]]=1
                    else: 
                        if A==1:
                            self.ic[self.i+3]=0
                        else:
                            self.ic[self.ic[self.i+3]]=0  
                else:
                    print('error')
            if opcode in (1,2,7,8):
                self.i+=4
            elif opcode in (3,4):
                self.i+=2 
        return 'error'
    
    def reset(self,ic):
        self.ic=ic
        self.i=0
        return

In [5]:
IC_org=[3,8,1001,8,10,8,105,1,0,0,21,46,59,72,93,110,191,272,353,434,99999,3,9,101,4,9,9,1002,9,3,9,1001,9,5,9,102,2,9,9,1001,9,5,9,4,9,99,3,9,1002,9,5,9,1001,9,5,9,4,9,99,3,9,101,4,9,9,1002,9,4,9,4,9,99,3,9,102,3,9,9,101,3,9,9,1002,9,2,9,1001,9,5,9,4,9,99,3,9,1001,9,2,9,102,4,9,9,101,2,9,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,99,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,99,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,99]
IC_ex1=[3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
IC_ex2=[3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0]
IC_ex3=[3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]
################################
IC_ex4=[3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
IC_ex5=[3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]

In [6]:
instructions=list(permutations([4,3,2,1,0]))
len(instructions)

120

In [7]:
outputs={}
#instruction=instructions[0]  
for instruction in instructions:

    # Amp A
    ICA=IC_org.copy()
    (outputA,ICA)=Incode(ICA,[instruction[0],0])
    # Amp B
    ICB=IC_org.copy()
    (outputB,ICB)=Incode(ICB,[instruction[1],outputA[0]])
    # Amp C
    ICC=IC_org.copy()
    (outputC,ICC)=Incode(ICC,[instruction[2],outputB[0]])
    # Amp D
    ICD=IC_org.copy()
    (outputD,ICD)=Incode(ICD,[instruction[3],outputC[0]])
    # Amp E
    ICE=IC_org.copy()
    (outputE,ICE)=Incode(ICE,[instruction[4],outputD[0]])

    outputs[instruction]=(outputE[0])
max(outputs.values())

21000

Part2: Feedback mode
Run once for initiation with modes: 5-9.
Then keep sending 'signal' form A to B to C to D to E to A ...
Don't restart the Amplifier Controller Software on any amplifier during this process. Each one should continue receiving and sending signals until it halts.
Start with 0
Eventually, the software on the amplifiers will halt after they have processed the final loop. When this happens, the last output signal from amplifier E is sent to the thrusters.
find the largest output signal that can be sent to the thrusters



In [8]:
instructions2=list(permutations([5,6,7,8,9]))

In [9]:
outputs=pd.DataFrame()
#instructions2=[(9,8,7,6,5)]
k=0
#instruction=instructions[0]  

ICA=IC_org.copy()
ICB=IC_org.copy()
ICC=IC_org.copy()
ICD=IC_org.copy()
ICE=IC_org.copy()


ampA=Amplifier(ICA)
ampB=Amplifier(ICB)
ampC=Amplifier(ICC)
ampD=Amplifier(ICD)
ampE=Amplifier(ICE)

for instruction in instructions2:
    output={'A':0,'B':0,'C':0,'D':0,'E':0}
    #initiate:
    #print('ampA')
    output['A']=ampA.start_up([instruction[0],0])
    #print('ampB')
    output['B']=ampB.start_up([instruction[1],output['A']])
    #print('ampC')
    output['C']=ampC.start_up([instruction[2],output['B']])
    #print('ampD')
    output['D']=ampD.start_up([instruction[3],output['C']])
    #print('ampE')
    output['E']=ampE.start_up([instruction[4],output['D']])
    
    outputs=outputs.append({'instruction':instruction,'loopnr':k,'outputA':output['A'],'outputB':output['B'],
                                'outputC':output['C'],'outputD':output['D'],'outputE':output['E']},ignore_index=True)
    
    #update:
    while output['E'] !='Done':
        #print('ampA')
        output['A']=ampA.update(output['E'])
        #print('ampB')
        output['B']=ampB.update(output['A'])
        #print('ampC')
        output['C']=ampC.update(output['B'])
        #print('ampD')
        output['D']=ampD.update(output['C'])
        #print('ampE')
        output['E']=ampE.update(output['D'])
        
        k+=1
        outputs=outputs.append({'instruction':instruction,'loopnr':k,'outputA':output['A'],'outputB':output['B'],
                                'outputC':output['C'],'outputD':output['D'],'outputE':output['E']},ignore_index=True)
   
    #reset amps
    ICA=IC_org.copy()
    ICB=IC_org.copy()
    ICC=IC_org.copy()
    ICD=IC_org.copy()
    ICE=IC_org.copy()

    ampA.reset(ICA)
    ampB.reset(ICB)
    ampC.reset(ICC)
    ampD.reset(ICD)
    ampE.reset(ICE)


In [10]:

del ampA
del ampB
del ampC
del ampD
del ampE

In [11]:
outputs.replace('Done',0,inplace=True)

In [12]:
np.max(outputs['outputE'])

61379886.0

In [13]:
outputs[outputs['outputE']==27601223]

,instruction,loopnr,outputA,outputB,outputC,outputD,outputE


In [14]:
outputs[outputs['instruction']==(8, 7, 6, 5, 9)]

,instruction,loopnr,outputA,outputB,outputC,outputD,outputE
946,"(8, 7, 6, 5, 9)",860.0,0.0,2.0,4.0,8.0,9.0
947,"(8, 7, 6, 5, 9)",861.0,11.0,12.0,14.0,28.0,29.0
948,"(8, 7, 6, 5, 9)",862.0,31.0,32.0,33.0,66.0,68.0
949,"(8, 7, 6, 5, 9)",863.0,136.0,272.0,273.0,546.0,1092.0
950,"(8, 7, 6, 5, 9)",864.0,2184.0,2186.0,4372.0,8744.0,17488.0
951,"(8, 7, 6, 5, 9)",865.0,34976.0,69952.0,69954.0,139908.0,139909.0
952,"(8, 7, 6, 5, 9)",866.0,139910.0,279820.0,559640.0,559642.0,559643.0
953,"(8, 7, 6, 5, 9)",867.0,559645.0,1119290.0,2238580.0,2238582.0,4477164.0
954,"(8, 7, 6, 5, 9)",868.0,8954328.0,8954329.0,17908658.0,17908660.0,35817320.0
955,"(8, 7, 6, 5, 9)",869.0,35817322.0,35817324.0,35817325.0,35817327.0,35817328.0


In [15]:
outputs

,instruction,loopnr,outputA,outputB,outputC,outputD,outputE
0,"(5, 6, 7, 8, 9)",0.0,0.0,2.0,4.0,8.0,9.0
1,"(5, 6, 7, 8, 9)",1.0,18.0,20.0,21.0,23.0,24.0
2,"(5, 6, 7, 8, 9)",2.0,48.0,49.0,50.0,52.0,54.0
3,"(5, 6, 7, 8, 9)",3.0,108.0,109.0,218.0,436.0,872.0
4,"(5, 6, 7, 8, 9)",4.0,1744.0,3488.0,3490.0,6980.0,13960.0
5,"(5, 6, 7, 8, 9)",5.0,27920.0,27922.0,55844.0,111688.0,111689.0
6,"(5, 6, 7, 8, 9)",6.0,111691.0,223382.0,446764.0,446765.0,446766.0
7,"(5, 6, 7, 8, 9)",7.0,446768.0,893536.0,1787072.0,1787074.0,3574148.0
8,"(5, 6, 7, 8, 9)",8.0,3574150.0,7148300.0,7148301.0,14296602.0,28593204.0
9,"(5, 6, 7, 8, 9)",9.0,28593206.0,28593207.0,28593209.0,28593211.0,28593212.0


In [6]:
ICA=ICB=ICC=ICD=ICE=[]
for IC in [ICA,ICB,ICC,ICD,ICE]:
    IC=IC_ex4.copy()

In [1]:
outputs

NameError: name 'outputs' is not defined